<a href="https://colab.research.google.com/github/Ahmed-Laaziz/NLP-Text-Generation/blob/main/gpt2_fineTuned_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 87.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.5 MB/s eta 0:00:00


In [2]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 6.3 MB/s eta 0:00:00


In [3]:
import json
def preprocess_intents_json(intents_file):
    with open(intents_file, "r") as f:
        data = json.load(f)

    preprocessed_data = []

    for intent in data["intents"]:
        for pattern in intent["patterns"]:
            preprocessed_data.append(f"User: {pattern}\n")
            for response in intent["responses"]:
                preprocessed_data.append(f"Assistant: {response}\n")

    return "".join(preprocessed_data)

def save_preprocessed_data(preprocessed_data, output_file):
    with open(output_file, "w") as f:
        f.write(preprocessed_data)

intents_file = "/content/drive/MyDrive/intents.json"
output_file = "/content/drive/MyDrive//mental_health_data.txt"

preprocessed_data = preprocess_intents_json(intents_file)
save_preprocessed_data(preprocessed_data, output_file)

In [4]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Config
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline

In [5]:
def fine_tune_gpt2(model_name, train_file, output_dir):
    # Load GPT-2 model and tokenizer
    model = GPT2LMHeadModel.from_pretrained(model_name)
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Load training dataset
    train_dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=train_file,
        block_size=128)
    # Create data collator for language modeling
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False)
    # Set training arguments
    training_args = TrainingArguments(
        output_dir=output_dir,
        overwrite_output_dir=True,
        num_train_epochs=50,
        per_device_train_batch_size=4,
        save_steps=20_000,
        save_total_limit=10,
    )
    # Train the model
    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset,
    )
    trainer.train()
    # Save the fine-tuned model
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)

In [6]:
# Fine-tune the model
fine_tune_gpt2("gpt2", "/content/drive/MyDrive/qa_pairs.txt", "output_dir_gpt2_fineTuned_model_abstracts")

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,3.466000
1000,3.247500
1500,3.035300
2000,2.821100
2500,2.770800
3000,2.544100
3500,2.478600
4000,2.405900
4500,2.219400
5000,2.196400


In [7]:
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/output_dir_gpt2_fineTuned_model_abstracts")
model_config = GPT2Config.from_pretrained("/content/drive/MyDrive/output_dir_gpt2_fineTuned_model_abstracts/config.json")
model = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/output_dir_gpt2_fineTuned_model_abstracts/pytorch_model.bin", config=model_config)


In [12]:
# Create a text generation pipeline
text_generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Generate text using the fine-tuned model
prompt = "Q: Can you give me an abstract for my research paper with the Title:Data science history?"
generated_text = text_generator(prompt, max_length=200, num_return_sequences=1)

# Print the generated text
print(generated_text[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Q: Can you give me an abstract for my research paper with the Title:Data science history?
A: Abstract:  Deception detection is an interdisciplinary field attracting researchers from
psychology, criminology, computer science, and economics. We propose a
multimodal approach combining deep learning and existing datasets to address 1910
requirements of data science education. Using the methods developed in
the field, we propose a curriculum that covers various subjects related to
identification, including social engineering, statistics, communication,
machine learning, and information systems biology. The proposed curriculum
consists of three parts: pre-trained language models and models designed according to
specifications by a black-box. The models are tested through an extensive
experimentation with synthetic data generated from a large language model comprising audio
and video modalities. The outcomes reveal that the proposed method can
achieve comparable accuracy to deep learning and 